In [14]:
import keras
from keras.activations import relu
from keras.callbacks import TensorBoard, EarlyStopping
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from keras.losses import MSE
from keras.metrics import mse
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam
import boto3
import pandas as pd
from tempfile import  NamedTemporaryFile
from librosa.feature import melspectrogram
import librosa
import numpy as np
import time
import requests
import json

In [4]:
boto_session = boto3.session.Session(region_name='eu-west-3')
songs_csv_path = 'https://s3.eu-west-3.amazonaws.com/sukikana-msd/full_dataset_10k.csv'


In [5]:
meta_csv_path = '../../data/sample_training_meta.csv'
song_factors_csv_path = '../../data/sample_song_factors.csv'

In [6]:
df_meta = pd.read_csv(meta_csv_path, sep=';')
df_song_factors = pd.read_csv(song_factors_csv_path, sep=';', index_col='song_id')

In [5]:
df_meta

,artist_id,artist_name,audio_md5,song_hotttnesss,song_id,title,track_id,preview_id,remote_preview_url,preview_artist,preview_title,preview_service
0,AROSQW61187FB5330B,The Fleetwoods,e4f5bce4689ffa1699ada608072400cf,0.310792,SOMSPXD12A8C139F36,Ten Times Blue,TRCCCDI128F427CC1F,tra.667543,http://listen.vo.llnwd.net/g1/4/3/3/1/8/104178...,The Fleetwoods,Ten Times Blue,napster
1,ARNZ70R1187B9A4824,Joe Satriani,0cf69352b3f246028cffc4a3b9fe9a4c,0.693272,SOSIYAD12A8C14097F,It's So Good,TRCCCMQ128F42AE752,tra.9456287,http://listen.vo.llnwd.net/g1/7/7/5/8/8/107288...,Joe Satriani,It's So Good,napster
2,ARCIUWR1269FCD7D0A,LANDA DANIEL,5c9d8c785695c084d209cdb60ec127e9,NaN,SOHPHTP12A8C13BF53,A KDO TEDA VODDELAL TOHO FANDU?,TRCCCFZ128F4283A22,tra.56658653,http://listen.vo.llnwd.net/g2/2/1/8/0/9/904190...,Landa Daniel,A kdo teda voddelal toho Fandu?,napster
3,ARYPORX1187FB52FBD,Nickodemus,294315b57b121eebdf71d7e234d4b1c7,0.690676,SOHXIRQ12AAA15CF81,Endangered Species,TRCCCSK128F92EE3B2,tra.8780127,http://listen.vo.llnwd.net/g3/3/6/2/7/7/133397...,Nickodemus,Endangered Species (feat. Marcelino Galan),napster
4,AR4TMIS1187FB52CE1,Tyrese,7c1d6460c4c83b59a727dedad2a0608b,0.734211,SOJHDEN12AB018B650,Gotta Get You,TRCCCEW128F42AF457,tra.12744134,http://listen.vo.llnwd.net/g1/6/0/7/2/2/105022...,Tyrese,Gotta Get You,napster
5,AR7KA5V1187FB44E6B,Coleman Hawkins,15995af9705aa67e5338b92925d2e0b0,0.494490,SONXGGO12A6D4F9CE5,Too Much Of A Good Thing,TRCCCZV128F932B336,tra.151614463,http://listen.vo.llnwd.net/g3/7/3/2/4/9/108829...,Coleman Hawkins,Too Much of a Good Thing,napster
6,ARP4A611187B9B8A37,Louis Prima,34d32f99a07eb901d3e6bbb5a850e4bb,0.000000,SOFDRQI12A6D4F9EBC,Tiger Rag,TRCCCLP128F426106F,tra.69972909,http://listen.vo.llnwd.net/g2/2/7/5/3/9/964093...,Louis Prima,Tiger Rag/Just Because (Live),napster
7,AR2SGQW1187B9B9E7C,KRS-One,398cf87756b63202c2e1b54fc2c51c1b,0.525497,SONVBWO12AB0187B35,Dream,TRCCCME128F9321775,tra.6189223,http://listen.vo.llnwd.net/g1/9/9/4/7/3/101293...,KRS-One,Dream,napster
8,ARH0NCL1187B99B41A,Quadro Nuevo,e6a66875a823ca857ed77f8a9c9b26b8,0.430161,SOPVMHL12A6D4F9B57,O Sarracino,TRCCCEJ128F14ACB97,tra.231355812,http://listen.vo.llnwd.net/g3/7/2/2/4/9/127239...,Quadro Nuevo,O sarracino,napster
9,ARABWZP1187FB41040,Vypsana Fixa,d31da8c2333e3da68ad830a10718420a,0.537273,SOGLAKB12AB017DF39,Kostel>Biohazard>MCD,TRCCCSV128F93006C3,tra.69456202,http://listen.vo.llnwd.net/g3/9/8/2/9/3/140993...,Vypsaná fixa,"Kostel, biohazard, McD",napster


In [6]:
df_song_factors

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
song_id,,,,,,,,,,,,,,,,,,,,,
SOAAGKQ12A8C13433E,0.003274,0.003953,0.003059,0.003896,0.003628,0.003678,0.003464,0.003770,0.003232,0.003103,...,0.003278,0.003256,0.003455,0.003569,0.003564,0.003171,0.003894,0.003189,0.003450,0.003381
SOAAPVS12A6D4F74B8,0.003746,0.004512,0.003565,0.004504,0.004281,0.004262,0.004350,0.004263,0.003877,0.003715,...,0.003953,0.003685,0.004185,0.004239,0.003988,0.003762,0.004499,0.003662,0.004339,0.003796
SOAAQXG12AB017E149,0.011688,0.006303,0.013275,0.010593,0.002179,0.008735,0.002234,0.008471,0.000800,0.015460,...,0.002597,0.012829,0.006741,0.016501,0.015245,0.001432,0.006752,0.012016,0.000497,0.008087
SOAAWPR12A8C13AE17,0.003410,0.004124,0.003306,0.004068,0.004133,0.003964,0.004004,0.003970,0.003632,0.003330,...,0.003691,0.003454,0.003940,0.003917,0.003870,0.003430,0.004145,0.003461,0.004116,0.003557
SOAAWRC12A6D4F77B2,0.003466,0.004178,0.003238,0.004288,0.003983,0.003873,0.003927,0.003905,0.003566,0.003257,...,0.003791,0.003514,0.004030,0.003906,0.003773,0.003408,0.004237,0.003348,0.004036,0.003568
SOAAYXD12A58A79881,0.003369,0.004118,0.003302,0.004100,0.004008,0.003819,0.003992,0.004078,0.003654,0.003473,...,0.003613,0.003534,0.003887,0.003908,0.003888,0.003725,0.004499,0.003492,0.004225,0.003589
SOAAZTZ12A8C131D62,0.004547,0.007368,0.004594,0.007113,0.006547,0.007994,0.007639,0.005025,0.004135,0.003796,...,0.007146,0.004418,0.003037,0.008353,0.006538,0.006955,0.009243,0.004554,0.001212,0.008165
SOABESB12A8C1358D5,0.003074,0.003880,0.002895,0.003726,0.003580,0.003490,0.003586,0.003673,0.003209,0.003088,...,0.003246,0.003144,0.003441,0.003510,0.003517,0.003166,0.003828,0.003153,0.003567,0.003124
SOABITL12A58A7D326,0.005158,0.006550,0.005254,0.006327,0.005718,0.005581,0.005785,0.005991,0.005233,0.005123,...,0.005218,0.005449,0.005771,0.006081,0.005861,0.005155,0.006662,0.005271,0.005814,0.005027


In [111]:
s3 = boto_session.resource('s3')
sukikana_bucket = 'sukikana'


def get_logmels(song_id, n_samples=8):
    with NamedTemporaryFile('wb+') as download_file:
        s3.Object(sukikana_bucket, song_id).download_fileobj(download_file)
        download_file.seek(0)
        offset = 0  # np.random.randint(26 * 1000) / 1000.0
        y, sr = librosa.load(download_file.name, offset=offset, sr=22050)
        mels = melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=512, power=2)
        min_mel = np.min(mels[np.where(mels != 0)])
        mels[np.where(mels == 0)] = min_mel / 10
        return np.log10(mels)


In [10]:
api = 'http://localhost:3099/api/v1/songs'

In [112]:
def save_logmels():
    for i, song in df_meta.loc[to_fix].iterrows():
        req_get = requests.get(api, params={'query': json.dumps({'msdId': song.song_id})}, timeout=5)# song.song_id})
        req_get.raise_for_status()
        res_get = req_get.json()[0]
        mels = get_logmels(res_get['msdId'])
        req_update = requests.patch(api + '/' + res['_id'], json={
            'mels': mels.reshape(-1).tolist(),
            'nComponents': mels.shape[0]
        })
        req_update.raise_for_status()

In [81]:
to_fix = np.array([147,
585,
621,
641,
663,
718,
758,
761,
791,
813,
879,
1115,
1138,
1212,
1354,
1359,
1784,
1821,
1855,
1943,
2087,
2155,
2199,
2218,
2259,
2309,
2378,
2493,
2784,
2799,
2976,
3102,
3177,
3208,
3273,
3409,
3424,
3493,
3631,
3654,
3675,
3984,
3985,
3993,
4123,
4280,
4681,
4689,
4708,
4796,
5055,
5104,
5507,
5632,
5676,
5699,
5705,
5800,
5870,
5887,
5964,
6026,
6147,
6357,
6575,
6590,
6621,
6639,
6774,
6816,
6848,
6953,
7004,
7012,
7308,
7344,
7359,
7470,
7564,
7720,
7745,
7884,
7940,
7942,
8079,
8094,
8153,
8189,
8241,
8359,
8401,
8408,
8491,
8590,
8797,
8875,
9117,
9169,
9384,
9486,
9530,
9696,
9720,
9796,
9811,
9983])

In [113]:
save_logmels()

In [44]:
req_get = requests.get(api, params={'query': json.dumps({'msdId': 'SOAAGKQ12A8C13433E'})}, timeout=5)# song.song_id})
req_get.raise_for_status()
res_get = req_get.json()[0]

In [46]:
mels = get_logmels(res_get['msdId'])

In [77]:
mels.shape

(128, 1292)

In [75]:
mels.reshape(-1).tolist()

array([-6.60195706, -4.28532786, -4.35667689, ..., -8.37697361,
       -8.52441072, -9.12847721])

In [76]:
req_update = requests.patch(api + '/' + res['_id'], json={'mels': mels.reshape(-1).tolist()})
req_update.raise_for_status()

(2, 1292)

527

In [25]:
req_update.raise_for_status()

HTTPError: 413 Client Error: Payload Too Large for url: http://localhost:3099/api/v1/songs/5b3d475a53b252b3760e50a6

In [18]:
res = req.json()[0]

In [ ]:
for _, song in df_meta.iterrows():
    req = requests.get(api, params={'msdId': song.song_id})

In [11]:
def build_supervised_training_set(df_song_factors, n_samples=3):
    mels = []
    factors = []
    for song_id in df_song_factors.index.values:
        try:
            song_mels = get_logmels(song_id, n_samples)
            for i in range(n_samples):
                mels.append(song_mels[i])
                factors.append(df_song_factors.loc[song_id].values)
        except:
            print(song_id)
    return mels, factors
        

In [9]:
mels, factors = build_supervised_training_set(df_song_factors[:500], n_samples=1)

In [12]:
mels, factors = build_supervised_training_set(df_song_factors, n_samples=1)

/Users/pascalluttgens/.pyenv/versions/anaconda3-4.4.0/envs/sukikana-content-based-model/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log10
  


SOLUYIQ12AF72A9BF4
SOMCPMK12A81C21A88
SOMERSX12AC9E16E38
SOTKHCU12AC9DCF57C
SOUTYUO12A58A7E4D5
SOXQCRS12AB0185373


In [10]:
np.save('../../data/training_mels', mels)

In [11]:
n_components = 128
mel_spectrum_length = 130
sampling_rate = 22050
n_factors = 50

In [12]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(4, 4), strides=2, use_bias=True, activation=relu, input_shape=(n_components, mel_spectrum_length, 1)))
model.add(Flatten())
model.add(Dense(500, activation=relu))
model.add(Dense(n_factors, activation=relu))
model.compile(optimizer=RMSprop(),loss=MSE, metrics=[mse])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 63, 64, 64)        1088      
_________________________________________________________________
flatten_1 (Flatten)          (None, 258048)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               129024500 
_________________________________________________________________
dense_2 (Dense)              (None, 50)                25050     
Total params: 129,050,638
Trainable params: 129,050,638
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(x=mels.reshape(mels.shape[0], mels.shape[1], mels.shape[2], 1), y=factors, validation_split=0.3, batch_size=32, epochs=50, callbacks=[TensorBoard(log_dir='./tensorboard-logs/run-{}'.format(time.strftime('%Y/%m/%d %H:%M:%S'))),])


Train on 350 samples, validate on 150 samples
Epoch 1/50
350/350 [==============================] - 20s 58ms/step - loss: 20.4876 - mean_squared_error: 20.4876 - val_loss: 4.3043e-05 - val_mean_squared_error: 4.3043e-05
Epoch 2/50
350/350 [==============================] - 19s 54ms/step - loss: 8.5495e-05 - mean_squared_error: 8.5495e-05 - val_loss: 4.3043e-05 - val_mean_squared_error: 4.3043e-05
Epoch 3/50
350/350 [==============================] - 19s 54ms/step - loss: 8.5495e-05 - mean_squared_error: 8.5495e-05 - val_loss: 4.3043e-05 - val_mean_squared_error: 4.3043e-05
Epoch 4/50
350/350 [==============================] - 21s 60ms/step - loss: 8.5495e-05 - mean_squared_error: 8.5495e-05 - val_loss: 4.3043e-05 - val_mean_squared_error: 4.3043e-05
Epoch 5/50
350/350 [==============================] - 21s 61ms/step - loss: 8.5495e-05 - mean_squared_error: 8.5495e-05 - val_loss: 4.3043e-05 - val_mean_squared_error: 4.3043e-05
Epoch 6/50
350/350 [==============================] - 26s 73

KeyboardInterrupt: 